<img src="./pics/c9e59b2f85ae246ccdca754c1e71ce69.jpg"  width="100" height= "100" align = "right"/>
# <font color=darkblue>DSCI 521 Project</font>
## <font color=darkred>*Musical Features Prediction Using Machine Learning Algorithms* </font>
#### <font color= darkgreen>*Using Audio Data Extracted From Free Music Archive Website* <br /> </font>
[Kiana Montazeri](https://github.com/kianamon)<sup>1</sup>, [Farnaz Ghashami](https://github.com/FarnazGhashami)<sup>1</sup>,
[Shideh Shams Amiri](https://github.com/shidehsh)<sup>1</sup>.<br>
<sup>1</sup>[Drexel University, Philadelphia, PA](https://drexel.edu/cci/academics/information-science-department/)


<img src="./pics/2.jpg"  width="900" height= "300" align = "center"/>

## Genre Prediction

In [1]:
#Libraries in Use
from pprint import pprint
from collections import defaultdict
import re
import csv
import pandas as pd
import IPython.display as ipd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, StandardScaler
from functools import reduce
import seaborn as sns

#### Load the data:

In [2]:
df_features = pd.read_csv("./Data/prepdata/Stat_Features.csv", low_memory=False)

In [3]:
parse_dates = ['date_created_x', 'date_released']
df_info = pd.read_csv("./Data/prepdata/info.csv", parse_dates = parse_dates)

In [4]:
# create the Labelencoder object
le = LabelEncoder()

#convert the categorical columns into numeric
df_info['genre_top'] = le.fit_transform(df_info['genre_top'])
df_info['tags_x'] = le.fit_transform(df_info['tags_x'])
df_info['members'] = le.fit_transform(df_info['members'])
df_info['name'] = le.fit_transform(df_info['name'])
df_info['tags'] = le.fit_transform(df_info['tags'])
df_info['tags_y'] = le.fit_transform(df_info['tags_y'])
df_info['license'] = le.fit_transform(df_info['license'])
df_info['information'] = le.fit_transform(df_info['information'])
df_info['date_created_x'] = le.fit_transform(df_info['date_created_x'])
df_info['date_released'] = le.fit_transform(df_info['date_released'])

In [5]:
df_info.dtypes

Unnamed: 0        int64
trackID           int64
bit_rate          int64
comments_x        int64
date_created_x    int64
duration          int64
favorites_x       int64
genre_top         int64
interest          int64
license           int64
listens_x         int64
number            int64
tags_x            int64
comments_y        int64
date_released     int64
favorites_y       int64
id_x              int64
information       int64
listens_y         int64
tags_y            int64
tracks            int64
favorites         int64
id_y              int64
members           int64
name              int64
tags              int64
dtype: object

In [6]:
df_info.head()

,Unnamed: 0,trackID,bit_rate,comments_x,date_created_x,duration,favorites_x,genre_top,interest,license,...,id_x,information,listens_y,tags_y,tracks,favorites,id_y,members,name,tags
0,0,2,256000,0,82,168,2,7,4656,36,...,1,1644,6073,2387,7,9,1,11914,299,1236
1,1,3,256000,0,83,237,1,7,1470,36,...,1,1644,6073,2387,7,9,1,11914,299,1236
2,2,5,256000,0,84,206,6,7,1933,36,...,1,1644,6073,2387,7,9,1,11914,299,1236
3,3,10,192000,0,0,161,178,12,54881,24,...,6,11053,47632,2387,2,74,6,7780,7425,10645
4,4,20,256000,0,86,311,0,7,978,24,...,4,10724,2710,2387,13,10,4,9984,9558,6377


In [7]:
#assigning the genre column as target
target = df_info['genre_top']

### Mel Frequency Cepstral Coefficients (MFCCs):

<font color=darkblue>The mel frequency cepstral coefficients (MFCCs) of a signal are a small set of features (usually about 10-20) which concisely describe the overall shape of a spectral envelope. In sound processing, the mel-frequency cepstrum is a representation of the short-term power spectrum of a sound, based on a linear cosine transform of a log power spectrum on a nonlinear mel scale of frequency. MFCCs are also increasingly finding uses in music information retrieval applications such as genre classification, audio similarity measures. So here, we are trying to extract the mfcc feature of the audio files for Genre prediction.</font>

In [8]:
colList = list(df_features.columns)
r = re.compile(".*mfcc")
mfcc_col_list = list(filter(r.match, colList)) # Read Note
print(mfcc_col_list)

['mfcc', 'mfcc.1', 'mfcc.2', 'mfcc.3', 'mfcc.4', 'mfcc.5', 'mfcc.6', 'mfcc.7', 'mfcc.8', 'mfcc.9', 'mfcc.10', 'mfcc.11', 'mfcc.12', 'mfcc.13', 'mfcc.14', 'mfcc.15', 'mfcc.16', 'mfcc.17', 'mfcc.18', 'mfcc.19', 'mfcc.20', 'mfcc.21', 'mfcc.22', 'mfcc.23', 'mfcc.24', 'mfcc.25', 'mfcc.26', 'mfcc.27', 'mfcc.28', 'mfcc.29', 'mfcc.30', 'mfcc.31', 'mfcc.32', 'mfcc.33', 'mfcc.34', 'mfcc.35', 'mfcc.36', 'mfcc.37', 'mfcc.38', 'mfcc.39', 'mfcc.40', 'mfcc.41', 'mfcc.42', 'mfcc.43', 'mfcc.44', 'mfcc.45', 'mfcc.46', 'mfcc.47', 'mfcc.48', 'mfcc.49', 'mfcc.50', 'mfcc.51', 'mfcc.52', 'mfcc.53', 'mfcc.54', 'mfcc.55', 'mfcc.56', 'mfcc.57', 'mfcc.58', 'mfcc.59', 'mfcc.60', 'mfcc.61', 'mfcc.62', 'mfcc.63', 'mfcc.64', 'mfcc.65', 'mfcc.66', 'mfcc.67', 'mfcc.68', 'mfcc.69', 'mfcc.70', 'mfcc.71', 'mfcc.72', 'mfcc.73', 'mfcc.74', 'mfcc.75', 'mfcc.76', 'mfcc.77', 'mfcc.78', 'mfcc.79', 'mfcc.80', 'mfcc.81', 'mfcc.82', 'mfcc.83', 'mfcc.84', 'mfcc.85', 'mfcc.86', 'mfcc.87', 'mfcc.88', 'mfcc.89', 'mfcc.90', 'mfcc.91',

In [9]:
mfcc_col_list.append('feature') #feature has trackID information, we will clean it later

In [10]:
df_mfcc = df_features[mfcc_col_list]

In [11]:
df_mfcc = df_mfcc.drop(df_mfcc.index[[0, 1, 2]])

In [12]:
df_mfcc = df_mfcc.rename(columns={'feature': 'trackID'})

In [13]:
df_mfcc.head()

,mfcc,mfcc.1,mfcc.2,mfcc.3,mfcc.4,mfcc.5,mfcc.6,mfcc.7,mfcc.8,mfcc.9,...,mfcc.131,mfcc.132,mfcc.133,mfcc.134,mfcc.135,mfcc.136,mfcc.137,mfcc.138,mfcc.139,trackID
3,3.8567891121e+00,1.5419009924e+00,8.1644812599e-04,3.3072751760e-01,1.1873064935e-01,-3.4268733859e-01,-2.5925174356e-01,1.4673459530e-01,4.1065633297e-01,-1.6287201643e-01,...,8.6019420624e+00,9.2842502594e+00,9.2455158234e+00,8.5208625793e+00,8.5604715347e+00,7.6518712044e+00,7.2465553284e+00,7.0771875381e+00,7.3918590546e+00,2
4,4.2967548370e+00,1.3999768496e+00,1.1253461987e-01,-2.1117024124e-01,3.2953105867e-02,-2.3488676175e-02,1.5040439367e-01,4.6454448253e-02,3.3484123647e-02,-6.4595922828e-02,...,1.0026866913e+01,6.9785413742e+00,7.6504168510e+00,9.6003570557e+00,7.2228884697e+00,8.3982934952e+00,7.2854228020e+00,7.4177908897e+00,8.7774400711e+00,3
5,2.6245169640e+00,2.4152929783e+00,4.4023323059e-01,-7.8213083744e-01,-7.7106910944e-01,-7.2421586514e-01,9.0259902179e-02,1.5211921930e-01,2.6173117757e-01,-6.0890531540e-01,...,9.5819520950e+00,8.8957233429e+00,8.1414556503e+00,8.2018442154e+00,7.7809634209e+00,7.1326918602e+00,7.5397529602e+00,8.4525270462e+00,7.3344416618e+00,5
6,5.0768928528e+00,1.1618540287e+00,2.0956509113e+00,1.3727425337e+00,-2.0357376337e-01,-3.4535357356e-01,-5.2913892269e-01,5.6197351217e-01,2.8135043383e-01,-1.5067242086e-01,...,7.9851098061e+00,7.0753998756e+00,6.9726486206e+00,7.0713930130e+00,7.2709589005e+00,7.0510702133e+00,6.9285907745e+00,6.4304733276e+00,6.1862936020e+00,10
7,1.1880131721e+01,4.0853204727e+00,2.6549834292e-03,1.5185616016e+00,1.8194729090e-01,3.3901408315e-01,3.6761111021e-01,6.9932222366e-02,-1.6031917185e-02,2.6011556387e-02,...,6.4989037514e+00,6.8455138206e+00,6.9344205856e+00,7.0193977356e+00,6.9838414192e+00,6.8136477470e+00,7.5208110809e+00,7.0980010033e+00,7.0322456360e+00,20


In [14]:
df_mfcc.shape

(106574, 141)

In [15]:
df_info.shape

(106574, 26)

In [16]:
#split data set into train and test sets
data_train, data_test, target_train, target_test = train_test_split(
    df_mfcc, target, test_size = 0.30, random_state = 10)

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
# Make an instance of the Model
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression(max_iter=1000)

In [19]:
logisticRegr.fit(data_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [20]:
predictions = logisticRegr.predict(data_test)

In [21]:
# Use score method to get accuracy of model
score = logisticRegr.score(data_test, target_test)
print(score)

0.6014449691927564


**<font color=green>There is an improvement to previous hypothesis but still there is room for a better model.</font>**

<font color=darkblue>Now, I want to use the whole feature data for Genre classification instead of only mfcc characteristics. In order to do so, let's prepare the whole dataset.</font>

In [22]:
df_features.head()

,Unnamed: 0,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,0,statistics,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,kurtosis,...,std,std,std,kurtosis,max,mean,median,min,skew,std
1,1,number,01,02,03,04,05,06,07,08,...,04,05,06,01,01,01,01,01,01,01
2,2,track_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
4,4,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02


In [23]:
features = df_features.drop(df_features.index[[0, 1, 2]])

In [24]:
# Make an instance of the Model
# all parameters not specified are set to their defaults
logReg = LogisticRegression(max_iter=1000)

In [25]:
#split data set into train and test sets
data_train2, data_test2, target_train2, target_test2 = train_test_split(
    features, target, test_size = 0.30, random_state = 10)

In [26]:
logReg.fit(data_train2, target_train2) #slow! :-S

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [27]:
predictions2 = logReg.predict(data_test2)

In [28]:
# Use score method to get accuracy of model
score2 = logReg.score(data_test2, target_test2)
print(score2)

0.6144559472054546


**<font color=green>We have the accuracy of which is a slight improvement compared to previous model. However the run time was much longer than the previous model. (Never mind 😨)</font>**

#### Test the model for my song:

In [31]:
ahaykhabardar = pd.read_csv("./Data/ahaykhabardar.csv")

In [32]:
ahaykhabardar.head()

,Unnamed: 0,Unnamed: 0.1,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,3,737373737373737,1,-0.595208,-0.790701,-0.651629,-0.56621,-0.2063,0.797669,-1.035652,...,0.128328,0.025725,0.02421,38.642639,0.471191,0.034443,0.026855,0.0,5.540561,0.036301


In [33]:
ahaykhabardar.drop('Unnamed: 0', axis=1, inplace=True)

In [34]:
ahaykhabardar.rename(columns={'Unnamed: 0.1': 'Unnamed: 0'}, inplace=True)

In [35]:
songahaykhabardar = ahaykhabardar.fillna(0)

In [36]:
df_mfcc_songahaykhabardar = songahaykhabardar[mfcc_col_list]

In [37]:
prediction_for_my_songahaykhabardar = logisticRegr.predict(df_mfcc_songahaykhabardar)

In [38]:
prediction_for_my_songahaykhabardar

array([11])

In [39]:
df_info2 = pd.read_csv("./Data/prepdata/info.csv", parse_dates = parse_dates)

In [40]:
le.fit_transform(df_info2['genre_top'])

array([ 7,  7,  7, ..., 13, 13,  7])

In [41]:
le_genre_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_genre_mapping)

{'Blues': 0, 'Classical': 1, 'Country': 2, 'Easy Listening': 3, 'Electronic': 4, 'Experimental': 5, 'Folk': 6, 'Hip-Hop': 7, 'Instrumental': 8, 'International': 9, 'Jazz': 10, 'Old-Time / Historic': 11, 'Pop': 12, 'Rock': 13, 'Soul-RnB': 14, 'Spoken': 15}


**<font color=green>So the closest genre for the random chosen song is Old-Time/Historic!</font>**

**Now let's try the model for another song:**

In [49]:
karaninadarad = pd.read_csv("./Data/karaninadarad.csv")
karaninadarad.head()

,Unnamed: 0,Unnamed: 0.1,feature,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
0,2,737373737373737,0,-1.281837,-0.949361,-1.323908,-1.369294,-0.818135,1.477261,-0.66137,...,0.158375,0.029285,0.026213,29.097673,0.415039,0.045508,0.037598,0.00293,4.427715,0.033465


In [50]:
karaninadarad.drop('Unnamed: 0', axis=1, inplace=True)

In [51]:
karaninadarad.rename(columns={'Unnamed: 0.1': 'Unnamed: 0'}, inplace=True)

In [52]:
songkaraninadarad = karaninadarad.fillna(0)

In [53]:
df_mfcc_songkaraninadarad = songkaraninadarad[mfcc_col_list]

In [54]:
prediction_for_my_songkaraninadarad = logisticRegr.predict(df_mfcc_songkaraninadarad)

In [55]:
prediction_for_my_songkaraninadarad

array([7])

**<font color=green>So the closest genre for the random chosen song is Hip-Hop!</font>**